<a href="https://colab.research.google.com/github/augusto1jr/PsyCare-Backend/blob/main/PsyCare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Imports**

In [1]:
!pip install scikit-surprise &> /dev/null

In [2]:
import pandas as pd
import numpy as np
import random

In [3]:
# Recomendação Baseada em Conteúdo
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [4]:
# Recomendação Baseada em Filtro Colaborativo
from surprise import Reader, Dataset, SVD
from surprise.accuracy import rmse, mae
from surprise.model_selection import cross_validate
from collections import defaultdict

**Os dados a seguir foram obtidos por meio de um scraper [Acesso disponível em: <https://github.com/augusto1jr/PsyCare-Scraping>] que extraiu os dados da plataforma Psicologia Viva [Acesso disponível em: <https://www.psicologiaviva.com.br/>].**

In [5]:
# Criando o dataframe que vai armazenar os dados dos psicólogos
psychologists = pd.read_csv('https://raw.githubusercontent.com/augusto1jr/PsyCare-Backend/main/psychologists.csv', low_memory=False)

Preenche com 'None' os dados não essenciais para os fins do projeto.

In [6]:
psychologists = psychologists.fillna('None')

As informações estão divididas em: **name** (nome do psicólogo), **bio** (biografia escrita pelo psicólogo), **expertises** (transtornos e
temáticas que o psicólogo possui expertise para oferecer tratamento), **total_ratings** (número total de avaliações recebidas pelo psicólogo), **average_rating** (nota média do psicólogo na plataforma).

In [7]:
psychologists

,name,bio,expertises,total_ratings,avarage_rating
0,Eliane Aguiar Monteiro Marques Cerchiari,"Olá! Como vc está?, Meu nome é Eliane e sou ps...","Acompanhamento pessoal, Ansiedade, Conflitos A...",269,5.000000
1,Rafael Cristiano Batista,"Olá, Como você está?, , Primeiramente, quero d...","Ansiedade, Casal, Casamento, Compulsões, Confl...",111,4.783784
2,Jhennifer Campos Lubanco de Barcelos,"Olá, bem vindo(a) ao meu perfil aqui na Psicol...","Acompanhamento psicológico de Gravidez, Ansied...",94,4.968085
3,Gustavo Simões Ramos,"Oi, me chamo Gustavo! Estou disponível para at...","Ansiedade, Casamento, Conflitos Amorosos, Conf...",174,4.988506
4,Fernanda Dudek,"Você que chegou até meu perfil, seja muito bem...","Acompanhamento psicológico de Gravidez, Acompa...",55,5.000000
...,...,...,...,...,...
1024,Mariana Lima Cavalcante Gaia do Couto,Me chamo Mariana e sou Psicóloga pela PUC-Rio....,"Ansiedade, Compulsões, Conflitos Amorosos, Con...",2,5.000000
1025,Francielly Ap Inocencio,"Meu nome é Francielly Inocencio, tenho 33 anos...","Ansiedade, Casal, Conflitos Amorosos, Conflito...",4,5.000000
1026,Thais De Vincenzo Schultheisz,Sou Dra. em Psicologia da Saúde. Atuo há mais ...,"Ansiedade, Depressão, Dor, Estresse, Família, ...",332,4.716867
1027,Lucas Ranyere Alves Costa,"Viver não é fácil, eu sei e você também sabe, ...","Ansiedade, Conflitos Amorosos, Conflitos Famil...",120,4.975000


# **Recomendação Baseada em Conteúdo**

Nesse cenário o algoritmo TF-IDF foi aplicado no campo *expertises* (especialidades de tratamento do psicólogo), criando uma matriz de similaridade que permite identificar o grau de similaridade entre os psicólogos da base de dados.

A partir disso é possível localizar um psicólogo na matriz de similaridade, e então ordenar os psicólogos com maior grau de similaridade para com o psicólogo em questão e exibir a recomendação gerada.


In [8]:
def Recomendacao_Baseada_Conteudo_Nome(name):
  # Calcula a matriz de similaridades entre os psicólogos
  vectorizer = TfidfVectorizer()
  similarities_matrix = vectorizer.fit_transform(psychologists['expertises'])
  similarity = linear_kernel(similarities_matrix, similarities_matrix)

  # Encontra o Psicólogo pelo nome
  psychologist = psychologists.loc[psychologists['name']==name]

  # Determina a similaridade relativa ao psicólogo em questão
  psy_similarity = list(enumerate(similarity[psychologist.index[0]]))

  # Recria a base de dados com o campo de similaridade anterior
  similar_psychologists = psychologists.copy()
  similar_psychologists['similarity'] = pd.Series(psy_similarity).apply(lambda x: x[1])

  # Ordena os psicólogos com maior similaridade em relação ao psicólogo em questão
  recommendations = sorted(psy_similarity, key=lambda x: x[1], reverse=True)

  # Seleciona os 10 psicólogos mais semelhantes
  '''O psicólogo de índice 0 é "pulado" pois sempre será o psicólogo em questão
  cujo se deseja estabelecer a similaridade aos outros (haverá 100% de similaridade com ele)'''
  recommendations = recommendations[1:11]

  # Seleciona os índices dos 10 psicólogos mais semelhantes
  recommendations_indexes = [i[0] for i in recommendations]

  # Retorna os 10 psicólogos mais semelhantes
  return similar_psychologists.iloc[recommendations_indexes]

In [10]:
Recomendacao_Baseada_Conteudo_Nome('Rafael Cristiano Batista')

,name,bio,expertises,total_ratings,avarage_rating,similarity
764,Amanda Caldas Cunha Cordeiro,"Olá, seja bem-vindo(a) ao meu perfil! Meu nome...","Acompanhamento pessoal, Ansiedade, Conflitos A...",46,5.000000,0.666882
168,Karine Vieira Matos,"Olá,, Meu nome é Karine, conhecida como Ká! So...","Ansiedade, Conflitos Amorosos, Depressão, Dese...",1,5.000000,0.641843
564,Karen Gomes,"Como psicóloga clínica, minha busca esta na re...","Acompanhamento pessoal, Ansiedade, Compulsões,...",26,5.000000,0.631617
447,Monique Carvalho,"Olá, como você está?, , Se você chegou até aqu...","Ansiedade, Casal, Casamento, Conflitos Amoroso...",91,4.978022,0.606228
324,Tatiane Macedo,Você sente que trabalha muito e não consegue o...,"Acompanhamento pessoal, Ansiedade, Casal, Conf...",45,5.000000,0.598624
59,Lainá Ferreira de Souza,"Olá, seja bem vindo (a) ao meu perfil! Estou a...","Acompanhamento psicológico de idosos, Ansiedad...",498,5.000000,0.598369
21,VITOR JUNIOR DA SILVA PINTO,"Olá, seja bem-vindo(a)! Se você chegou até o m...","Ansiedade, Conflitos Amorosos, Conflitos Famil...",0,0.000000,0.593656
669,Márcia Maria Tavares de Souza,Atendimento Clínico Autoconhecimento-o ser int...,"Conflitos Familiares, Desenvolvimento Pessoal,...",28,5.000000,0.593461
702,Ester Salema,"Olá, tudo bem? Seja bem vindo (a) ao meu consu...","Acompanhamento pessoal, Ansiedade, Casamento, ...",79,5.000000,0.584010
254,Isabela Mendes,"Olá! Seja bem vinda (o), auxilio pessoas a lid...","Ansiedade, Casal, Conflitos Amorosos, Conflito...",80,4.937500,0.581982


No exemplo acima a recomendação foi gerada ao estabelecer a similaridade entre um psicólogo em questão com relação à outros.

Além disso, também foi implementada uma outra forma de recomendação baseada em conteúdo que não necessita do parâmetro de um psicólogo. Nesse cenário, o usuário deve inserir alguns problemas que tem interesse em receber tratamento.

In [11]:
def Recomendacao_Baseada_Conteudo_Problemas(problems):
  # Cria um Psicólogo temporário
  '''Esse psicólogo possui como especialidades os problemas que o usuário
  inseriu. Ele vai funcionar como um "psicólogo ideal" e servir como base
  para encontrar o psicólogo mais semelhante com o que o usuário procura.'''
  psychologists.loc[len(psychologists)] = ['temporary psychologist', None, problems, None, None]
  temp_psychologist = psychologists.loc[psychologists['name']=='temporary psychologist']

  # Calcula a matriz de similaridades entre os psicólogos
  vectorizer = TfidfVectorizer()
  similarities_matrix = vectorizer.fit_transform(psychologists['expertises'])
  similarity = linear_kernel(similarities_matrix, similarities_matrix)

  # Determina a similaridade relativa ao psicólogo em questão
  psy_similarity = list(enumerate(similarity[temp_psychologist.index[0]]))

  # Recria a base de dados com o campo de similaridade anterior
  similar_psychologists = psychologists.copy()
  similar_psychologists['similarity'] = pd.Series(psy_similarity).apply(lambda x: x[1])

  # Ordena os psicólogos com maior similaridade em relação ao psicólogo em questão
  recommendations = sorted(psy_similarity, key=lambda x: x[1], reverse=True)

  # Seleciona os 10 psicólogos mais semelhantes
  '''Nesse caso são incluídos 11 psicólogos pois o
  psicólogo temporário será excluído posteriormente'''
  recommendations = recommendations[0:11]

  # Seleciona os índices dps 10 psicólogos mais semelhantes
  recommendations_indexes = [i[0] for i in recommendations]

  # Remove o psicólogo temporário
  psychologists.drop(temp_psychologist.index, inplace=True)
  recommendations_indexes.remove(temp_psychologist.index)

  # Retorna os 10 psicólogos mais semelhantes
  return similar_psychologists.iloc[recommendations_indexes]

In [13]:
# Nesse caso foram usados as especialidades (campo expertises) do psicólogo Rafael Cristiano Batista
# com o objetivo de avaliar a precisão do algoritmo, que corretamente apresentou 1.0 (100%) de similaridade com o mesmo

Recomendacao_Baseada_Conteudo_Problemas('Ansiedade, Casal, Casamento, Compulsões, Conflitos Amorosos, Conflitos Familiares, Dependência Química, Depressão, Desenvolvimento Pessoal, Estresse, Família, Orientação de pais, Orientação Profissional, Psicologia Clínica, Psicologia Organizacional, Saúde Mental, Violência Sexual, Sexualidade e identidade de gênero, Seleção de pessoal por competências, Sexualidade e Identidade de gênero, Maternidade, Paternidade')

,name,bio,expertises,total_ratings,avarage_rating,similarity
1,Rafael Cristiano Batista,"Olá, Como você está?, , Primeiramente, quero d...","Ansiedade, Casal, Casamento, Compulsões, Confl...",111,4.783784,1.000000
764,Amanda Caldas Cunha Cordeiro,"Olá, seja bem-vindo(a) ao meu perfil! Meu nome...","Acompanhamento pessoal, Ansiedade, Conflitos A...",46,5.000000,0.666302
168,Karine Vieira Matos,"Olá,, Meu nome é Karine, conhecida como Ká! So...","Ansiedade, Conflitos Amorosos, Depressão, Dese...",1,5.000000,0.641211
564,Karen Gomes,"Como psicóloga clínica, minha busca esta na re...","Acompanhamento pessoal, Ansiedade, Compulsões,...",26,5.000000,0.630732
447,Monique Carvalho,"Olá, como você está?, , Se você chegou até aqu...","Ansiedade, Casal, Casamento, Conflitos Amoroso...",91,4.978022,0.606194
59,Lainá Ferreira de Souza,"Olá, seja bem vindo (a) ao meu perfil! Estou a...","Acompanhamento psicológico de idosos, Ansiedad...",498,5.000000,0.599007
324,Tatiane Macedo,Você sente que trabalha muito e não consegue o...,"Acompanhamento pessoal, Ansiedade, Casal, Conf...",45,5.000000,0.597837
21,VITOR JUNIOR DA SILVA PINTO,"Olá, seja bem-vindo(a)! Se você chegou até o m...","Ansiedade, Conflitos Amorosos, Conflitos Famil...",0,0.000000,0.594441
669,Márcia Maria Tavares de Souza,Atendimento Clínico Autoconhecimento-o ser int...,"Conflitos Familiares, Desenvolvimento Pessoal,...",28,5.000000,0.592697
702,Ester Salema,"Olá, tudo bem? Seja bem vindo (a) ao meu consu...","Acompanhamento pessoal, Ansiedade, Casamento, ...",79,5.000000,0.583241


# **Recomendação Baseada em Filtragem Colaborativa**

**Os dados a seguir também foram obtidos por meio do mesmo scraper [Acesso disponível em: <https://github.com/augusto1jr/PsyCare-Scraping>] que extraiu os dados da plataforma Psicologia Viva [Acesso disponível em: <https://www.psicologiaviva.com.br/>].**

As informações estão divididas em: **patient** (paciente que realizou a avaliação), **psychologist** (psicólogo que recebeu a avaliação), **rating** (nota dada pelo paciente ao psicólogo).

In [15]:
# Criando o dataframe que vai armazenar os dados de avaliações
ratings = pd.read_csv('https://raw.githubusercontent.com/augusto1jr/PsyCare-Backend/main/ratings.csv', low_memory=False)

In [16]:
# O sistema de avaliações funciona com base em consultas
# Dessa forma, um paciente pode avaliar diversas vezes o mesmo psicólogo
# Portanto, é necessário remover as linhas que representam avaliações repetidas
ratings = ratings.drop_duplicates()

In [17]:
ratings

,patient,psychologist,rating
0,O,Eliane Aguiar Monteiro Marques Cerchiari,5
1,GBS,Eliane Aguiar Monteiro Marques Cerchiari,5
2,KRGDF,Eliane Aguiar Monteiro Marques Cerchiari,5
3,SBSDF,Eliane Aguiar Monteiro Marques Cerchiari,5
5,SPGB,Eliane Aguiar Monteiro Marques Cerchiari,5
...,...,...,...
93017,FCDMS,Matheus Rodrigues do Nascimento,5
93024,KKC,Matheus Rodrigues do Nascimento,5
93027,RDOV,Matheus Rodrigues do Nascimento,5
93031,FFRDM,Matheus Rodrigues do Nascimento,5


In [18]:
# Inserindo manualmente um usuário com suas avaliações no dataframe
# Nesse cenário o paciente avaliou bem um grupo de psicólogos semelhantes entre si
# E avaliou mal os psicólogos mais diferentes com relação ao grupo

# O paciente possui intencionalmente poucas avaliações
# O objetivo é testar possíveis problemas de cold start posteriormente
ratings = pd.concat([ratings, pd.DataFrame.from_records([{'patient': 'AAAAA', 'psychologist': 'Isabela Rodrigues Leal', 'rating': 5}])], ignore_index=True)
ratings = pd.concat([ratings, pd.DataFrame.from_records([{'patient': 'AAAAA', 'psychologist': 'Herika Blumer de Vita Bezerra da Silva', 'rating': 5}])], ignore_index=True)
ratings = pd.concat([ratings, pd.DataFrame.from_records([{'patient': 'AAAAA', 'psychologist': 'Rafael Franco', 'rating': 5}])], ignore_index=True)
ratings = pd.concat([ratings, pd.DataFrame.from_records([{'patient': 'AAAAA', 'psychologist': 'Glauco Luiz Targino', 'rating': 5}])], ignore_index=True)
ratings = pd.concat([ratings, pd.DataFrame.from_records([{'patient': 'AAAAA', 'psychologist': 'Jairla Santana dos Santos', 'rating': 5}])], ignore_index=True)
ratings = pd.concat([ratings, pd.DataFrame.from_records([{'patient': 'AAAAA', 'psychologist': 'Jamile Bangemann', 'rating': 5}])], ignore_index=True)
ratings = pd.concat([ratings, pd.DataFrame.from_records([{'patient': 'AAAAA', 'psychologist': 'Amanda Manis Casarini', 'rating': 4}])], ignore_index=True)
ratings = pd.concat([ratings, pd.DataFrame.from_records([{'patient': 'AAAAA', 'psychologist': "Fernanda D'Avila Rodrigues", 'rating': 3}])], ignore_index=True)
ratings = pd.concat([ratings, pd.DataFrame.from_records([{'patient': 'AAAAA', 'psychologist': 'Ana Cristina Wagner', 'rating': 2}])], ignore_index=True)
ratings = pd.concat([ratings, pd.DataFrame.from_records([{'patient': 'AAAAA', 'psychologist': 'Caroline Santos de Almeida Martins', 'rating': 1}])], ignore_index=True)


## **Treinando o modelo**

Para a recomendação colaborativa o objetivo foi utilizar os dados de avaliações descritos acima para prever as notas que o paciente daria para os psicólogos, isso foi feito utilizando o SVD (Singular Value Decomposition) em conjunto com o Cross-Validation, servindo como forma de treinamento e avaliação do modelo.

In [19]:
# Informa o range das notas, nesse sistema ele vai de 1 até 5
reader = Reader(rating_scale = (1, 5))

# Cria um dataset com Reader
data = Dataset.load_from_df(ratings[['patient', 'psychologist', 'rating']], reader)

In [20]:
# Criando instancia do SVD
np.random.seed(5)
svd = SVD()

# Treinando os dados
cross_validate(svd, data, measures = ['MAE', 'RMSE'], cv=5, verbose = True)
print('>> Traning Done')

Evaluating MAE, RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
MAE (testset)     0.1233  0.1288  0.1251  0.1240  0.1171  0.1237  0.0038  
RMSE (testset)    0.3741  0.4030  0.3904  0.3689  0.3322  0.3737  0.0240  
Fit time          0.38    0.34    0.37    0.34    0.38    0.36    0.02    
Test time         0.03    0.03    0.03    0.03    0.02    0.03    0.00    
>> Traning Done


In [21]:
# Cria a base de testes
trainset = data.build_full_trainset()

# Pega as notas que não estão no teste e preenche com a média
antiset = trainset.build_anti_testset()

In [22]:
# Gera todas as previsões
predictions = svd.test(antiset)

In [23]:
# Prevê a nota que um usuário aleatório daria para um psicólogo específico
svd.predict('GBS', 'Rafael Cristiano Batista')

Prediction(uid='GBS', iid='Rafael Cristiano Batista', r_ui=None, est=4.916057619194589, details={'was_impossible': False})

In [24]:
# Prevê a nota que o usuário inserido manualmente daria para um psicólogo específico
svd.predict('AAAAA', 'Rafael Cristiano Batista')

Prediction(uid='AAAAA', iid='Rafael Cristiano Batista', r_ui=None, est=4.348226193885154, details={'was_impossible': False})

## **Algoritmo de Filtragem Colaborativa**

A partir do treinamento realizado pelo Cross-Validation e do cálculo previsões pelo SVD, é possível gerar uma lista de recomendações com base nos psicólogos que receberam as maiores notas previstas.

In [25]:
# Para cada usuário, seleciona o top 10 psicólogos com a melhor nota prevista
top_predictions = defaultdict(list)
for u_id, p_id, _, est, _ in predictions:
    top_predictions[u_id].append((p_id,est))
for u_id, user_ratings in top_predictions.items():
    user_ratings.sort(key = lambda x: x[1], reverse = True)
    top_predictions[u_id] = user_ratings[:10]

In [29]:
def Recomendacao_Colaborativa(user):
  # Transforma o dicionário em um dataframe
  top_predictions_df = pd.DataFrame(top_predictions[user])
  top_predictions_df.columns = ['psychologist', 'predicted_rating']

  # Busca os psicólogos do top10 na base de dados
  recommendations = pd.DataFrame()
  for i in range(len(top_predictions_df)):
    prediction = top_predictions_df.loc[top_predictions_df.index == i].psychologist.item()
    psychologist = psychologists.loc[psychologists.name == prediction]
    recommendations = pd.concat([recommendations, psychologist[['name', 'bio', 'expertises', 'total_ratings', 'avarage_rating']]], ignore_index=True)

  recommendations['predicted_rating'] = top_predictions_df['predicted_rating']

  return recommendations

In [30]:
# Resultado da recomendação colaborativa com um usuário aleatório
Recomendacao_Colaborativa('GBS')

,name,bio,expertises,total_ratings,avarage_rating,predicted_rating
0,Fernanda Dudek,"Você que chegou até meu perfil, seja muito bem...","Acompanhamento psicológico de Gravidez, Acompa...",55,5.000000,5
1,Andressa Severo Pinto,"Olá, seja bem-vindo ao meu perfil. Meu nome é ...","Ansiedade, Compulsões, Conflitos Amorosos, Dep...",4,5.000000,5
2,Hellen Rose Correa Santos,None,"Ansiedade, Casal, Casamento, Conflitos Amoroso...",9,4.888889,5
3,Simony Almeida,"Avaliação, Diagnóstico e Tratamento em TDAH | ...","Ansiedade, Aprendizagem, Compulsões, Conflitos...",63,5.000000,5
4,ADRIANA REGINA JULIO,"Olá, primeiramente parabéns pela iniciativa de...","Ansiedade, Cirurgia Bariátrica, Compulsões, Co...",124,4.983871,5
5,Jessica Lais Soares Nunes da Silva,Atuo a quatro anos no atendimento clínico psic...,"Acompanhamento psicológico de Gravidez, Acompa...",9,5.000000,5
6,Luciane Pereira Maia Silveira,Meu nome é Luciane mas também conhecida como L...,"Acompanhamento pessoal, Ansiedade, Aprendizage...",20,4.950000,5
7,Hugo Luiz Araújo José,"Olá, seja bem-vindo(a)!, , Meu nome é Hugo Ara...","Ansiedade, Conflitos Familiares, Depressão, De...",3,5.000000,5
8,Lylian Santos e Silva,*** ENTRE EM CONTATO PARA VERIFICAR POSSIBILID...,"Ansiedade, Conflitos Familiares, Consultoria E...",317,4.996845,5
9,Carina Cruz de Santana Almeida,Trabalho em conjunto com os princípios psicana...,"Acompanhamento pessoal, Ansiedade, Casal, Casa...",24,5.000000,5


In [31]:
# Resultado da recomendação colaborativa com o usuário inserido manualmente
Recomendacao_Colaborativa('AAAAA')

,name,bio,expertises,total_ratings,avarage_rating,predicted_rating
0,Maysa Valbom Ladeira,"Oiii! Muito prazer!, Me chamo Maysa, sou psico...","Acompanhamento psicológico de idosos, Ansiedad...",3,5.000000,4.829982
1,Bruna Ribeiro Lima Barbosa,"Olá, seja bem-vindo(a). , Que bom te ver por a...","Acompanhamento pessoal, Ansiedade, Depressão, ...",9,5.000000,4.819084
2,Monica Freitas Pessanha Vieira,"Olá fica tranquilo, Psicoterapia é pra todos!!...","Ansiedade, Conflitos Familiares, Depressão, De...",96,5.000000,4.798524
3,Gustavo Brehmer,"- Atendo a partir de 18 anos., A vida se apres...","Acompanhamento pessoal, Ansiedade, Casamento, ...",384,5.000000,4.777459
4,Henrique Paixão,"Oi, eu sou o Henrique! Sou formado em Psicolog...","Ansiedade, Casal, Compulsões, Dependência Quím...",15,5.000000,4.762813
5,Monice Kattar Giovanni,"Sou psicóloga há 16 anos, professora universit...","Ansiedade, Conflitos Amorosos, Conflitos Famil...",25,5.000000,4.749837
6,TAINÁ KURTZ,Olá! Eu sou psicóloga e especializanda em Tera...,"Acompanhamento psicológico de Gravidez, Acompa...",251,5.000000,4.742987
7,Walace Gonçalves Veloso,"Olá, me chamo Walace Veloso, seja bem-vindo! E...","Ansiedade, Casamento, Conflitos Amorosos, Conf...",15,5.000000,4.740020
8,Andressa Mara Marques,Olá! Se você busca autoconhecimento; melhorar ...,"Ansiedade, Conflitos Amorosos, Conflitos Famil...",79,5.000000,4.724120
9,TAIRINI EVLIN CARVALHO,"Escutar, compreender seus sentimentos, angústi...","Ansiedade, Compulsões, Conflitos Amorosos, Con...",198,4.989899,4.720764


É possível observar que o usuário inserido manualmente possui uma recomendação razoavelmente pior, levando em consideração as notas dos psicólogos incluídos na recomendação. Esse é um problema de coldstart que ocorre devido ao menor número de dados de avaliações sobre esse usuário no sistema.

# **Recomendação Baseada em Filtragem Híbrida**

Por fim, o algoritmo de recomendação baseado em filtragem híbrida faz uma mescla entre os dois algoritmos citados anteriormente (TF-IDF e SVD) buscando gerar uma recomendação que balanceie os pontos fracos de ambos.

## **Modelo 1 (TFIDF-SVD)**

O primeiro modelo híbrido funciona inicialmente calculando a similaridade por TF-IDF, e posteriormente ordenando esse resultado com base na nota prevista pelo SVD.

In [32]:
# Recomendação Híbrida Ordenada por Nota Prevista

def Recomendacao_Hibrida_TFIDF_SVD(user, psy_name):
  # Calcula a matriz de similaridades entre os psicólogos
  vectorizer = TfidfVectorizer()
  similarities_matrix = vectorizer.fit_transform(psychologists['expertises'])
  similarity = linear_kernel(similarities_matrix, similarities_matrix)

  # Encontra o Psicólogo pelo nome
  psychologist = psychologists.loc[psychologists['name']==psy_name]
  psy_id = int(psychologist.index[0])

  # Determina a similaridade relativa ao psicólogo em questão
  psy_similarity = list(enumerate(similarity[psy_id]))

  # Recria a base de dados com o campo de similaridade anterior
  similar_psychologists = psychologists.copy()
  similar_psychologists['similarity_level'] = pd.Series(psy_similarity).apply(lambda x: x[1])

  # Ordena os psicólogos com maior similaridade em relação ao psicólogo em questão
  recommendations = sorted(psy_similarity, key=lambda x: x[1], reverse=True)

  # Seleciona os 100 psicólogos mais semelhantes
  '''O psicólogo de índice 0 é "pulado" pois sempre será o psicólogo em questão
  cujo se deseja estabelecer a similaridade aos outros (haverá 100% de similaridade com ele)'''
  recommendations = recommendations[1:101]

  # Seleciona os índices dos 100 psicólogos mais semelhantes
  recommendations_indexes = [i[0] for i in recommendations]

  # Busca os 100 psicólogos na base de dados
  similar_psychologists = similar_psychologists.iloc[recommendations_indexes]

  # Preve a nota do usuario para os 100 psicólogos mais semelhantes
  similar_psychologists['predicted_rating'] = similar_psychologists.name.apply(lambda x: svd.predict(user, x).est)

  # Ordena a recomendação com base na nota prevista
  similar_psychologists = similar_psychologists.sort_values('predicted_rating', ascending=False)

  return similar_psychologists[0:10]

In [33]:
# Resultado do primeiro modelo de recomendação híbrida com um usuário aleatório
Recomendacao_Hibrida_TFIDF_SVD('GBS', 'Rafael Cristiano Batista')

,name,bio,expertises,total_ratings,avarage_rating,similarity_level,predicted_rating
764,Amanda Caldas Cunha Cordeiro,"Olá, seja bem-vindo(a) ao meu perfil! Meu nome...","Acompanhamento pessoal, Ansiedade, Conflitos A...",46,5.000000,0.666882,5.0
852,Jurece Curupaná,"Meu nome é Jurecê de Oliveira Curupaná, sou Ps...","Ansiedade, Casal, Casamento, Cirurgia Bariátri...",166,5.000000,0.468418,5.0
132,Lorrayne Ismael,"Olá, me chamo Lorrayne, sou psicóloga, pós gra...","Acompanhamento psicológico de Gravidez, Acompa...",73,5.000000,0.537944,5.0
387,Eloize Marianny Bonfim da Silva,None,"Ansiedade, Casal, Conflitos Amorosos, Conflito...",14,5.000000,0.471770,5.0
325,Camilla Schultz,"Olá, meu nome é Camilla e sou psicóloga clínic...","Ansiedade, Casamento, Conflitos Amorosos, Conf...",121,4.991736,0.528079,5.0
141,Andreza da silva andrade,"Olá! me chamo Andreza, tenho 26 anos. Me forme...","Ansiedade, Casal, Casamento, Conflitos Amoroso...",67,4.955224,0.525794,5.0
744,Caroline Orlande Gabriel,Sou graduada em Psicologia pela Universidade F...,"Acompanhamento pessoal, Ansiedade, Casamento, ...",19,5.000000,0.521512,5.0
248,Fernanda Gomes de Melo Lima,Uma professora ao longo da minha formação um d...,"Ansiedade, Compulsões, Conflitos Amorosos, Con...",50,5.000000,0.519243,5.0
533,João Luiz Corrêa,"Olá! Como está se sentindo? , , Meu nome é Joã...","Acompanhamento pessoal, Ansiedade, Casamento, ...",88,5.000000,0.512659,5.0
623,Elisangela Lima de Paula,"Olá eu nome é Elisângela, me graduei no ano de...","Acompanhamento pessoal, Acompanhamento psicoló...",1316,0.436930,0.512489,5.0


In [34]:
# Resultado do primeiro modelo de recomendação híbrida com o usuário inserido manualmente
Recomendacao_Hibrida_TFIDF_SVD('AAAAA', 'Rafael Cristiano Batista')

,name,bio,expertises,total_ratings,avarage_rating,similarity_level,predicted_rating
58,Sandra Pinheiro,"“Olá, sou psicóloga há seis anos . Tenho exper...","Acompanhamento pessoal, Ansiedade, Conflitos A...",4,5.0,0.509941,4.673082
701,Beatriz De Oliveira Toral,"Olá, seja bem-vindo(a)! Sou Beatriz, psicóloga...","Ansiedade, Conflitos Amorosos, Conflitos Famil...",38,5.0,0.497528,4.667403
324,Tatiane Macedo,Você sente que trabalha muito e não consegue o...,"Acompanhamento pessoal, Ansiedade, Casal, Conf...",45,5.0,0.598624,4.646881
248,Fernanda Gomes de Melo Lima,Uma professora ao longo da minha formação um d...,"Ansiedade, Compulsões, Conflitos Amorosos, Con...",50,5.0,0.519243,4.629283
983,Fernanda Cristina Grilletti,"Meu nome é Fernanda Cristina Grilletti, CRP 06...","Ansiedade, Aprendizagem, Conflitos Amorosos, C...",4,5.0,0.499235,4.628978
94,Natalia Iara Alves Isepon,"Olá, sou psicóloga clínica e trabalho pautada ...","Acompanhamento psicológico de idosos, Ansiedad...",41,5.0,0.476238,4.627176
160,Caio Rangel,"Enquanto houver vida dentro de você, existirá ...","Ansiedade, Casal, Casamento, Conflitos Familia...",96,5.0,0.467188,4.627133
814,Renata Sousa Lima Ribeiro de Oliveira,"Psicóloga Clínica., Psicoterapeuta de Adolesc...","Acompanhamento pessoal, Ansiedade, Compulsões,...",104,5.0,0.498591,4.625746
744,Caroline Orlande Gabriel,Sou graduada em Psicologia pela Universidade F...,"Acompanhamento pessoal, Ansiedade, Casamento, ...",19,5.0,0.521512,4.611128
627,Fabiana Galla Soares dos Santos,"Olá, se você está aqui, possivelmente surgiu u...","Acompanhamento pessoal, Ansiedade, Casamento, ...",41,5.0,0.544767,4.610946


## **Modelo 2 (SVD-TFIDF)**

Já o segundo modelo híbrido funciona de maneira inversa, inicialmente calculando as notas previstas pelo SVD, e posteriormente ordenando esse resultado com base na similaridade por TF-IDF.

In [35]:
# Para cada usuário, seleciona o top 100 psicólogos com a melhor nota prevista
top_predictions = defaultdict(list)
for u_id, p_id, _, est, _ in predictions:
    top_predictions[u_id].append((p_id,est))
for u_id, user_ratings in top_predictions.items():
    user_ratings.sort(key = lambda x: x[1], reverse = True)
    top_predictions[u_id] = user_ratings[:100]

In [37]:
def Recomendacao_Hibrida_SVD_TFIDF(user, psy_name):
  # Transforma o dicionário em um dataframe
  top_predictions_df = pd.DataFrame(top_predictions[user])
  top_predictions_df.columns = ['psychologist', 'predicted_rating']

  # Busca os psicólogos do top10 na base de dados
  recommendations = pd.DataFrame()
  for i in range(len(top_predictions_df)):
    prediction = top_predictions_df.loc[top_predictions_df.index == i].psychologist.item()
    psychologist = psychologists.loc[psychologists.name == prediction]
    recommendations = pd.concat([recommendations, psychologist[['name', 'bio', 'expertises', 'total_ratings', 'avarage_rating']]], ignore_index=True)

  # Adiciona a coluna de notas previstas
  recommendations['predicted_rating'] = top_predictions_df['predicted_rating']

  # Calcula a matriz de similaridades entre os psicólogos
  vectorizer = TfidfVectorizer()
  similarities_matrix = vectorizer.fit_transform(recommendations['expertises'])
  similarity = linear_kernel(similarities_matrix, similarities_matrix)

  # Encontra o Psicólogo pelo nome
  psychologist = psychologists.loc[psychologists['name']==psy_name]

  # Determina a similaridade relativa ao psicólogo em questão
  psy_similarity = list(enumerate(similarity[psychologist.index[0]]))

  # Recria a base de dados com o campo de similaridade anterior
  similar_psychologists = recommendations.copy()
  similar_psychologists['similarity'] = pd.Series(psy_similarity).apply(lambda x: x[1])
  similar_psychologists = similar_psychologists.sort_values('similarity', ascending=False)

  return similar_psychologists[1:11]

In [38]:
# Resultado do primeiro modelo de recomendação híbrida com um usuário aleatório
Recomendacao_Hibrida_SVD_TFIDF('GBS', 'Rafael Cristiano Batista')

,name,bio,expertises,total_ratings,avarage_rating,predicted_rating,similarity
65,Isabelle telyne Contel Anzulim,"Olá, voce se sente ansioso? sem vontade de fa...","Ansiedade, Compulsões, Conflitos Amorosos, Dep...",51,4.980392,5,0.776750
48,Geiziane Silva Mota,"Olá! Sou Geiziane Silva Mota, Psicóloga Clínic...","Ansiedade, Conflitos Amorosos, Conflitos Famil...",21,4.857143,5,0.700091
76,Katiussia Viana da Silva,"Olá, me chamo Katiussia, e sou Psicóloga Clíni...","Ansiedade, Conflitos Amorosos, Depressão, Dese...",39,5.000000,5,0.648997
97,Henrique Lovato Corte Real,"Olá, seja bem-vindo(a)! , Sou psicólogo, ativo...","Ansiedade, Aprendizagem, Compulsões, Conflitos...",628,2.939490,5,0.597224
19,Henrique Paixão,"Oi, eu sou o Henrique! Sou formado em Psicolog...","Ansiedade, Casal, Compulsões, Dependência Quím...",15,5.000000,5,0.571970
64,Larissa Favarão Bianchesi,None,"Acompanhamento pessoal, Ansiedade, Casamento, ...",114,5.000000,5,0.548429
61,Franciele Miloch dos Santos,"Olá! Prazer, meu nome é Franciele e meu princi...","Ansiedade, Compulsões, Conflitos Familiares, D...",88,5.000000,5,0.547517
98,Luciene Martins,"Olá, Seja muito bem vindo (a)! Aqui, você va...","Ansiedade, Compulsões, Conflitos Amorosos, Con...",5,5.000000,5,0.544796
96,Kareen Cristina Centurion Jordão,"Como você vem se sentindo? Desanimado, ansioso...","Acompanhamento pessoal, Acompanhamento psicoló...",28,5.000000,5,0.528556
66,Caroline Rosa Aquino,"Atuo como Psicóloga Clínica, auxiliando jovens...","Ansiedade, Compulsões, Conflitos Amorosos, Con...",10,5.000000,5,0.515125


In [39]:
# Resultado do segundo modelo de recomendação híbrida com um usuário aleatório
Recomendacao_Hibrida_SVD_TFIDF('AAAAA', 'Rafael Cristiano Batista')

,name,bio,expertises,total_ratings,avarage_rating,predicted_rating,similarity
48,Cintia Regina Oliveira Macedo,"Seja bem vindo!, Sou psicóloga e psicanalista....","Ansiedade, Casal, Compulsões, Conflitos Famili...",84,4.988095,4.645162,0.696008
49,Aline Machado,Atuo como psicóloga utilizando a abordagem psi...,"Acompanhamento psicológico de idosos, Ansiedad...",225,5.000000,4.643292,0.659237
35,Ana Carolina Mendes Caixeta,Você tem se sentido mais ansioso e acredita qu...,"Ansiedade, Compulsões, Conflitos Amorosos, Con...",223,4.995516,4.659556,0.601992
38,Giovana Lourenço Crudi,Você quer melhorar sua qualidade de vida e de ...,"Ansiedade, Casamento, Conflitos Amorosos, Conf...",10,5.000000,4.654621,0.585399
63,Eliene Aparecida do Carmo,". Graduação em Psicologia, pela Estácio de Sá....","Ansiedade, Conflitos Familiares, Estresse Pós ...",65,4.984615,4.630526,0.576172
4,Henrique Paixão,"Oi, eu sou o Henrique! Sou formado em Psicolog...","Ansiedade, Casal, Compulsões, Dependência Quím...",15,5.000000,4.762813,0.575421
19,Larissa Favarão Bianchesi,None,"Acompanhamento pessoal, Ansiedade, Casamento, ...",114,5.000000,4.684741,0.567318
32,Fernanda Dudek,"Você que chegou até meu perfil, seja muito bem...","Acompanhamento psicológico de Gravidez, Acompa...",55,5.000000,4.663319,0.546246
28,Ana Luiza Pereira Amaral,"Olá, que bom te ter aqui! Buscar ajuda não cos...","Ansiedade, Conflitos Amorosos, Depressão, Dese...",111,5.000000,4.668393,0.533298
10,Delcia Gomes da Silva soares,"Olá: bem vindo-a, ao nosso espaço psicológico...","Acompanhamento psicológico de idosos, Ansiedad...",167,4.191617,4.718030,0.505049


No cenário de ambos os modelos híbridos é possível observar uma maior variedade entre os psicólogos recomendados quando comparados com a recomendação por conteúdo, mostrando a capacidade de mitigar o problema de falta de variedade desse tipo de recomendação.

Além disso, ambos os modelos também apresentaram uma melhora nos resultados do usuário inserido manualmente, mostrando a capacidade do modelo híbrido de mitigar o problema de cold start, que pode ter sido o motivo das notas mais baixas na recomendação colaborativa.

Ao comparar os dois modelos entre si, é possível observar que ambos apresentam resultados positivos em termos de notas previstas, apesar do modelo SVD-TFIDF apresentar resultados melhores nos níveis de similaridade.